In [1]:
import sys
sys.path.append('/home/zhangy/python_project/mms')  # 替换为 alpa_serve 的实际路径

from alpa_serve.trace import Trace

fixed_rate_scale = 2e-3
fixed_cv_scale = 10
trace_dir = "/home/zhangy/data/datasets/azurefunctions-dataset2019/azure_v1.pkl"
model_set = ["bert-6.7b", "moe-5.3b", "bert-2.6b", "moe-2.4b", "bert-1.3b", "moe-1.3b"]
fixed_num_modelset = 1
model_types = model_set * fixed_num_modelset
model_names = sum([[f"{model_type}-{i}" for model_type in model_set] for i in range(fixed_num_modelset)], [])

arrival_process_kwargs = {"rate_scale": fixed_rate_scale,
                          "cv_scale": fixed_cv_scale,
                          "trace_dir": trace_dir}

azure_v1_trace_dir = arrival_process_kwargs["trace_dir"]
azure_v1_trace = Trace("azure_v1", azure_v1_trace_dir)
train_replays = azure_v1_trace.replay(model_names,
                                        model_mapping_strategy="stripe",
                                        arrival_distribution="gamma",
                                        start_time="0.0.0",
                                        end_time="0.1.0",
                                        interval_seconds=3600,
                                        rate_scale_factor=arrival_process_kwargs["rate_scale"],
                                        cv_scale_factor=arrival_process_kwargs["cv_scale"])

Reading takes: 4.435631990432739s.
Sliced trace stats: #functions: 72359, total invocations: 32862030, max: 4880144, min: 0, avg: 454.15


In [2]:
for replay in train_replays:
    train_replays[replay].report_stats()
    train_replays[replay].visualize()

Trace for bert-6.7b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 9106, arrival distribution: gamma, generation interval: 3600, scale factor: (0.002, 10, 1.0, 1). overall rate: 2.53, overall cv: 9.23.
Trace for moe-5.3b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 8148, arrival distribution: gamma, generation interval: 3600, scale factor: (0.002, 10, 1.0, 1). overall rate: 2.26, overall cv: 10.24.
Trace for bert-2.6b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 10199, arrival distribution: gamma, generation interval: 3600, scale factor: (0.002, 10, 1.0, 1). overall rate: 2.83, overall cv: 10.23.
Trace for moe-2.4b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 10711, arrival distribution: gamma, generation interval: 3600, scale factor: (0.002, 10, 1.0, 1). overall rate: 2.98, overall cv: 9.01.
Trace for bert-1.3b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 11536, arrival distribution: gamma, generation interval: 3600, scale factor: (0.002, 10, 1.0, 1). overall rate: 3.20, 

In [32]:
from alpa_serve.simulator.workload import Workload, GammaProcess
from collections import OrderedDict
from alpa_serve.trace import Trace, TraceReplay
import numpy as np

rate_distribution = "power_law"
arrival_process = "gamma"
arrival_process_kwargs = {"cv": 4}
total_rate = 20
duration = 3600
interval_seconds = 600

# 设置峰值率和基础率
peak_rate = 2  # 峰值期间的请求率
base_rate = 0.2  # 其他时间的基础请求率

fixed_num_devices = 32
fixed_num_modelset = 1
fixed_slo_scale = 5

model_set = ["bert-6.7b", "moe-5.3b", "bert-2.6b", "moe-2.4b", "bert-1.3b", "moe-1.3b"]
model_types = model_set * fixed_num_modelset
model_names = sum([[f"{model_type}-{i}" for model_type in model_set] for i in range(fixed_num_modelset)], [])

num_models = len(model_names)

# 定义到达过程
arrival_processes = []
if rate_distribution == "power_law":
    alpha = 1
    s = sum((x+1)**(-alpha) for x in range(num_models))
    base = total_rate / s
    rates = [base * ((x+1) ** (-alpha)) for x in range(num_models)]
if arrival_process == "gamma":
    for i in range(num_models):
        # 设置波峰的时间段（例如：100-200秒和300-400秒）
        num_intervals = duration // interval_seconds
        # 随机选择波峰的时间段
        peak_times = np.random.choice(num_intervals, 2, replace=False)
        distribution = []
        for t in range(duration // interval_seconds):
            if t in peak_times:
                rate = peak_rate * rates[i]
            else:
                rate = base_rate * rates[i]
            distribution.append(GammaProcess(rate, arrival_process_kwargs["cv"]))
        arrival_processes.append(distribution)

replays = OrderedDict()
start_time = "0.0.0"
end_time = f"0.{int(duration / 3600)}.0"
start_d, start_h, start_m = Trace.timestr_to_dhm(start_time)
end_d, end_h, end_m = Trace.timestr_to_dhm(end_time)
start_timestamp_seconds = start_d * 24 * 60 * 60 + start_h * 60 * 60 + start_m * 60
end_timestamp_seconds = end_d * 24 * 60 * 60 + end_h * 60 * 60 + end_m * 60

seed = 0
for m in range(len(arrival_processes)):
    arrivals = []
    arrival_distribution_params = []
    for i, distribution in enumerate(arrival_processes[m]):
        if distribution is None:
            arrival_distribution_params.append(None)
            continue
        start = i * interval_seconds + start_timestamp_seconds
        arrivals.extend(distribution.generate_arrivals(start, interval_seconds, seed))
        # if DEBUG:
        #     arrivals.extend(distribution.generate_arrivals(0, 1.0e9, seed))
        #     self.visualize_inter_arrival(np.array(arrivals), "test")
        arrival_distribution_params.append(distribution.params())
        seed += 1
    replays[model_names[m]] = TraceReplay(model_names[m],
                             np.array(arrivals),
                             "synthetic",
                             start_time,
                             end_time,
                             interval_seconds,
                             arrival_distribution="power_law",
                             arrival_distribution_params=arrival_distribution_params)

In [33]:
for replay in replays:
    replays[replay].report_stats()
    replays[replay].visualize()

Trace for bert-6.7b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 22606, arrival distribution: power_law, generation interval: 600, scale factor: (1.0, 1.0, 1.0, 1). overall rate: 6.28, overall cv: 6.17.
Trace for moe-5.3b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 12663, arrival distribution: power_law, generation interval: 600, scale factor: (1.0, 1.0, 1.0, 1). overall rate: 3.52, overall cv: 6.92.
Trace for bert-2.6b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 8048, arrival distribution: power_law, generation interval: 600, scale factor: (1.0, 1.0, 1.0, 1). overall rate: 2.24, overall cv: 6.38.
Trace for moe-2.4b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 5811, arrival distribution: power_law, generation interval: 600, scale factor: (1.0, 1.0, 1.0, 1). overall rate: 1.61, overall cv: 6.99.
Trace for bert-1.3b-0, duration: (0, 1, 0, 0), 3600 (s), #arrivals: 4729, arrival distribution: power_law, generation interval: 600, scale factor: (1.0, 1.0, 1.0, 1). overall rate: